# Assessment Explorartory Analysis 
Computing some basic stats about the state of paratransit agencies, based on Oliva's work

In [1]:
import pandas as pd
import intake
import altair as alt
alt.themes.enable("urbaninstitute")

ThemeRegistry.enable('urbaninstitute')

In [2]:
catalog = intake.open_catalog('./catalog.yml')

In [3]:
df = catalog.assessment_sheet(csv_kwargs={'skiprows': [1]}).read()

In [4]:
print(f"The number of agencies assessed was {len(df)}")
print(f"Of these, {len(df[df.is_provider == True])} were found to be actual providers of paratransit service")

The number of agencies assessed was 217
Of these, 202 were found to be actual providers of paratransit service


In [5]:
# only subset to providers
df = df[df.is_provider == True]

Percentage of agencies that are mills act (TDA funded)

In [6]:
df.is_mills_act.value_counts(normalize=True)

True     0.831633
False    0.168367
Name: is_mills_act, dtype: float64

## Application Types

In [52]:
alt.Chart(pd.DataFrame(df.digital_app.value_counts(normalize=True).rename({'PDF_form': 'PDF Form',
                                                                           "no_online_app": "No Online Application",
                                                                           "standard_web_based_form": "Standard Web Form"})).reset_index()).mark_bar().encode(
    alt.X('digital_app', title='Percentage of Agencies', axis=alt.Axis(format='%')),
    alt.Y('index', title='Type of Application')
).properties(
    title='Percentage of Agencies taking Web Based Applications',
)

alt.Chart(...)

As you can see, a pluarity of agencies have no online application. 40% simply give you a PDF form that must then be submitted via paper or email. Finally, 10% take a standard, web based interactive form. 

## Payment Tyes

In [11]:
def split_payment_methods(row):
    """
    taking in the value of payments methods, split in 
    list
    """
    if row == 'none_fare_free':
        return row
    elif row == 'credit_card_bulk':
        return row
    elif "_" in row: 
        return row.split('_')
    else: 
        return row

In [12]:
df=df.assign(payments_methods=df.payment_methods.apply(split_payment_methods))

In [13]:
payments = {}
for item in df.payments_methods.values:
    if type(item) == list:
        for i in item: 
            if i not in payments:
                payments[i] = 0
            payments[i] += 1
    else:
        if item not in payments:
            payments[item] = 0
        payments[item] += 1

In [63]:
p_df = pd.DataFrame.from_dict(payments, orient='index').reset_index().rename({0: 'count', 'index': 'payment_type'}, axis=1).replace({'credit_card_bulk': "Credit card (bulk purchase)",
                                                                                                          'none_fare_free': "None (fare-free)"}).replace(
{'cash': 'Cash', 'check':'Check', 'credit':'Credit', 'other': 'Other'}
)
alt.Chart(p_df).mark_bar().encode(
  alt.X('count', title="Number of Agencies Taking Payment Form"),
  alt.Y('payment_type', title="payment type")
).properties(
    title='Agency Fare Payment Options',
)

alt.Chart(...)

# Agencies advertising cross-eligibility

We looked to see if the agencies post a notice about using another agencies elibility verification. 

In [64]:
src = df.fillna(False).replace({True: 'Yes', False: 'No'}).assign(is_mills_act=df.fillna(False).is_mills_act.replace({True: 'Mills Act Agency', False:"Not Mills Act Agency"}))[['cross_elibility', 'is_mills_act']]
alt.Chart(src).mark_bar().encode(
    alt.Y('cross_elibility', title=''),
    alt.X('count(cross_elibility)',title="Number of Agencies"),
    color=alt.Color('is_mills_act', title='Mills Act Agency')
).properties(
    title='Does the Agency advertise cross eligibility?',
)

alt.Chart(...)

As you can see, a minority of agencies are advertising cross-eligibility on their web / public site. 